# Capstone Project

### 1. Importing libraries

In [23]:
import pandas as pd # library for data analsysis
import json
import numpy as np
import codecs
import requests # library to handle requests
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
# For preprocessing
from sklearn.preprocessing import MinMaxScaler
!pip install folium==0.5 
import folium # map rendering library
import urllib.request
from bs4 import BeautifulSoup
!pip install geocoder
import geocoder
print('Libraries imported.')

Libraries imported.


In [24]:
### 2. Webscrap dataset from Weikipedia
url = 'https://en.wikipedia.org/wiki/List_of_boroughs_of_Oslo'
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")
all_tables=soup.find_all("table")
print(all_tables)
right_table=soup.find('table', class_='wikitable sortable')
print(right_table)
A=[]
B=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==4:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))


[<table class="wikitable sortable">
<tbody><tr>
<th>Borough</th>
<th>Residents</th>
<th>Area</th>
<th>Number
</th></tr>
<tr>
<td><a href="/wiki/Alna" title="Alna">Alna</a></td>
<td>49 801</td>
<td>13,7 km<sup>2</sup></td>
<td align="right">12
</td></tr>
<tr>
<td><a href="/wiki/Bjerke" title="Bjerke">Bjerke</a></td>
<td>33 422</td>
<td>7,7 km<sup>2</sup></td>
<td align="right">9
</td></tr>
<tr>
<td><a href="/wiki/Frogner" title="Frogner">Frogner</a></td>
<td>59 269</td>
<td>8,3 km<sup>2</sup></td>
<td align="right">5
</td></tr>
<tr>
<td><a href="/wiki/Gamle_Oslo" title="Gamle Oslo">Gamle Oslo</a></td>
<td>58 671</td>
<td>7,5 km<sup>2</sup></td>
<td align="right">1
</td></tr>
<tr>
<td><a href="/wiki/Grorud" title="Grorud">Grorud</a></td>
<td>27 707</td>
<td>8,2 km<sup>2</sup></td>
<td align="right">10
</td></tr>
<tr>
<td><a href="/wiki/Gr%C3%BCnerl%C3%B8kka" title="Grünerløkka">Grünerløkka</a></td>
<td>62 423</td>
<td>4,8 km<sup>2</sup></td>
<td align="right">2
</td></tr>
<tr>
<td><a hre

In [25]:
print(A)
print(B)

['Alna', 'Bjerke', 'Frogner', 'Gamle Oslo', 'Grorud', 'Grünerløkka', 'Nordre Aker', 'Nordstrand', 'Sagene', 'St. Hanshaugen', 'Stovner', 'Søndre Nordstrand', 'Ullern', 'Vestre Aker', 'Østensjø']
['49 801', '33 422', '59 269', '58 671', '27 707', '62 423', '52 327', '52 459', '45 089', '38 945', '33 316', '39 066', '34 596', '50 157', '50 806']


### 3. Set Forsquare API Details

In [26]:
CLIENT_ID = 'QYIGBRO3WT5ZOTKAJ2RGIUQKSWPEM0HIGG4P52NHIJUGSEKA' # your Foursquare ID
CLIENT_SECRET = 'U4ZYXK5FMXLCMT311EQQQBVLLNM3VBMDWCXVHKIABNBS5QB3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QYIGBRO3WT5ZOTKAJ2RGIUQKSWPEM0HIGG4P52NHIJUGSEKA
CLIENT_SECRET:U4ZYXK5FMXLCMT311EQQQBVLLNM3VBMDWCXVHKIABNBS5QB3


### 4. Create DataFrame using the webscrapped table of Boroughs and preprocess the number of residents to get 0-1 (minmax)

In [27]:
# instantiate the dataframe
df_oslo = pd.DataFrame(data = {'Borough': A, 'Residents' : B})
df_oslo = df_oslo.replace(r' ','', regex=True) 
df_oslo['Residents'].astype('int')
print(df_oslo)
#For normalization
scaler = MinMaxScaler()
scaler.fit(df_oslo[['Residents']])
df_oslo['Residents'] = scaler.transform(df_oslo[['Residents']])
df_oslo = df_oslo.sort_values(by='Residents',ascending = False).reset_index(drop=True)
df_oslo


             Borough Residents
0               Alna     49801
1             Bjerke     33422
2            Frogner     59269
3          GamleOslo     58671
4             Grorud     27707
5        Grünerløkka     62423
6         NordreAker     52327
7         Nordstrand     52459
8             Sagene     45089
9      St.Hanshaugen     38945
10           Stovner     33316
11  SøndreNordstrand     39066
12            Ullern     34596
13        VestreAker     50157
14          Østensjø     50806


,Borough,Residents
0,Grünerløkka,1.000000
1,Frogner,0.909149
2,GamleOslo,0.891923
3,Nordstrand,0.712985
4,NordreAker,0.709183
5,Østensjø,0.665370
6,VestreAker,0.646676
7,Alna,0.636421
8,Sagene,0.500691
9,SøndreNordstrand,0.327198


### 5. Get Lat and Longitude for every Borough and place them into two separe columns

In [28]:
# Defining a function to get coordinates
def get_latlng(borough):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Oslo, Norway'.format(borough))
        lat_lng_coords = g.latlng
    return lat_lng_coords
# Call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(borough) for borough in df_oslo['Borough'].tolist()]

In [29]:
column1 = [item[0] for item in coords]
column2 = [item[1] for item in coords]  

### 6. Add previously acquired columns to the df_oslo and add longitude and latitude of Oslo

In [30]:
print(column1)
df_oslo['Latitude'], df_oslo['Longitude'] = column1, column2
# Lat, Long of Oslo
latitude,longitude = 59.9139,10.7522

[59.924330000000054, 59.91886000000005, 59.90624000000008, 59.86075000000005, 59.94933000000003, 59.87992000000003, 59.94805000000008, 59.92680000000007, 59.93868000000003, 59.83342000000005, 59.92679000000004, 59.92186000000004, 59.942210000000046, 59.96279000000004, 59.96058000000005]


In [31]:
#Check outpout of df_oslo to see if columns were added correctly
df_oslo

,Borough,Residents,Latitude,Longitude
0,Grünerløkka,1.000000,59.92433,10.76119
1,Frogner,0.909149,59.91886,10.70239
2,GamleOslo,0.891923,59.90624,10.77147
3,Nordstrand,0.712985,59.86075,10.80260
4,NordreAker,0.709183,59.94933,10.73140
5,Østensjø,0.665370,59.87992,10.82584
6,VestreAker,0.646676,59.94805,10.64203
7,Alna,0.636421,59.92680,10.84570
8,Sagene,0.500691,59.93868,10.75763
9,SøndreNordstrand,0.327198,59.83342,10.80778


### 7. Visualize dataframe of Boroughs in the map os Oslo

In [32]:
# create map of Oslo using latitude and longitude values
map_oslo = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough in zip(df_oslo['Latitude'], df_oslo['Longitude'], df_oslo['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_oslo)  
map_oslo.save('index-oslo.html')
map_oslo

### 8. Get venues in the surrounding areas of the Boroughs

In [33]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
df_oslo_venues = getNearbyVenues(names=df_oslo['Borough'],
                                   latitudes=df_oslo['Latitude'],
                                   longitudes=df_oslo['Longitude'])

Grünerløkka
Frogner
GamleOslo
Nordstrand
NordreAker
Østensjø
VestreAker
Alna
Sagene
SøndreNordstrand
St.Hanshaugen
Ullern
Bjerke
Stovner
Grorud


In [35]:
print(df_oslo_venues.shape)
df_oslo_venues.head()

(296, 7)


,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Grünerløkka,59.92433,10.76119,Grünerløkka Brygghus,59.925018,10.759250,Pub
1,Grünerløkka,59.92433,10.76119,Oslovelo,59.925457,10.761000,Café
2,Grünerløkka,59.92433,10.76119,Bon Lio,59.922238,10.761378,Spanish Restaurant
3,Grünerløkka,59.92433,10.76119,Parkteatret,59.923403,10.758397,Performing Arts Venue
4,Grünerløkka,59.92433,10.76119,Focacceria,59.923632,10.757336,Italian Restaurant


### 9. See which borough has the highest amount of venues

In [36]:
df_oslo_venues.groupby('Borough').count()

,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
Alna,23,23,23,23,23,23
Bjerke,12,12,12,12,12,12
Frogner,21,21,21,21,21,21
GamleOslo,18,18,18,18,18,18
Grorud,10,10,10,10,10,10
Grünerløkka,72,72,72,72,72,72
NordreAker,19,19,19,19,19,19
Nordstrand,18,18,18,18,18,18
Sagene,32,32,32,32,32,32


Grunerlokka seems to be the best epicentre. Let's see what the clusters say

### 10. Create dummies for venue categories to see which one is most frequent per Borough

In [37]:
# one hot encoding
df_oslo_onehot = pd.get_dummies(df_oslo_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df_oslo_onehot['Borough'] = df_oslo_venues['Borough'] 

# move neighborhood column to the first column
first_col = df_oslo_onehot.pop("Borough")
df_oslo_onehot.insert(0,"Borough" , first_col)
df_oslo_onehot

,Borough,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bakery,Bar,Beer Garden,...,Sushi Restaurant,Thai Restaurant,Theme Park,Thrift / Vintage Store,Toy / Game Store,Tram Station,Video Store,Wine Bar,Wine Shop,Yoga Studio
0,Grünerløkka,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Grünerløkka,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Grünerløkka,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Grünerløkka,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Grünerløkka,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,Grorud,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
292,Grorud,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
293,Grorud,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
294,Grorud,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [38]:
df_oslo_grouped = df_oslo_onehot.groupby('Borough').mean().reset_index()
df_oslo_grouped

,Borough,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bakery,Bar,Beer Garden,...,Sushi Restaurant,Thai Restaurant,Theme Park,Thrift / Vintage Store,Toy / Game Store,Tram Station,Video Store,Wine Bar,Wine Shop,Yoga Studio
0,Alna,0.000000,0.043478,0.000000,0.000000,0.043478,0.000000,0.043478,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.043478,0.000000,0.000000,0.000000,0.043478,0.000000
1,Bjerke,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Frogner,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,...,0.047619,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000
3,GamleOslo,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.055556,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Grorud,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000
5,Grünerløkka,0.000000,0.013889,0.027778,0.000000,0.000000,0.013889,0.027778,0.083333,0.013889,...,0.000000,0.013889,0.00000,0.013889,0.000000,0.013889,0.000000,0.013889,0.000000,0.013889
6,NordreAker,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,...,0.052632,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000
7,Nordstrand,0.000000,0.000000,0.055556,0.055556,0.000000,0.000000,0.055556,0.000000,0.000000,...,0.055556,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Sagene,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.031250,0.000000,...,0.093750,0.000000,0.03125,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,St.Hanshaugen,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.096774,0.032258,0.000000,...,0.096774,0.032258,0.00000,0.000000,0.000000,0.000000,0.032258,0.000000,0.000000,0.000000


### 11. Get the top 10 most common venues per Borough

In [39]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
borough_venues_sorted = pd.DataFrame(columns=columns)
borough_venues_sorted['Borough'] = df_oslo_grouped['Borough']

for ind in np.arange(df_oslo_grouped.shape[0]):
    borough_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_oslo_grouped.iloc[ind, :], num_top_venues)

borough_venues_sorted

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alna,Electronics Store,Furniture / Home Store,Grocery Store,Supermarket,Department Store,Bus Station,Bookstore,Kids Store,Flower Shop,Sporting Goods Shop
1,Bjerke,Supermarket,Pizza Place,Soccer Field,Grocery Store,Asian Restaurant,Racetrack,Gym / Fitness Center,Chinese Restaurant,Bus Station,Shopping Mall
2,Frogner,Gourmet Shop,Italian Restaurant,Scandinavian Restaurant,Furniture / Home Store,Wine Shop,French Restaurant,Indian Restaurant,Movie Theater,Park,Cosmetics Shop
3,GamleOslo,Coffee Shop,Record Shop,Park,Café,Plaza,Cocktail Bar,Italian Restaurant,Pizza Place,Restaurant,Moving Target
4,Grorud,Convenience Store,Grocery Store,Wine Shop,Pizza Place,Bus Station,Supermarket,Metro Station,Soccer Field,Eastern European Restaurant,Cosmetics Shop
5,Grünerløkka,Coffee Shop,Bar,Park,Italian Restaurant,Burger Joint,Café,Cocktail Bar,Brewery,Pub,Asian Restaurant
6,NordreAker,Grocery Store,Bus Station,Supermarket,Rental Car Location,Metro Station,Shopping Mall,Electronics Store,Kids Store,Fast Food Restaurant,Bike Shop
7,Nordstrand,Grocery Store,Fast Food Restaurant,Gym,Supermarket,Coffee Shop,Restaurant,Bus Station,Shopping Mall,Juice Bar,Bookstore
8,Sagene,Café,Sushi Restaurant,Park,Bakery,Deli / Bodega,Coffee Shop,Bus Station,Dessert Shop,Skate Park,Fish Market
9,St.Hanshaugen,Coffee Shop,Sushi Restaurant,Bakery,Pizza Place,Café,Indian Restaurant,Park,Cocktail Bar,Deli / Bodega,Pub


It seems like coffee shops and bars are prominent in Grunerlokka

### 12. Find clusters and see which one is useful to solve our problem

In [40]:
# set number of clusters
kclusters = 5

df_oslo_grouped_clustering = df_oslo_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_oslo_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 1, 1, 2, 1, 2, 2, 1, 1], dtype=int32)

In [41]:
borough_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [42]:
df_oslo_merged = df_oslo
# merge df_oslo_grouped with df_oslo to add latitude/longitude for each neighborhood
df_oslo_merged = df_oslo_merged.join(borough_venues_sorted.set_index('Borough'), on='Borough')
df_oslo_merged # check the last columns!

,Borough,Residents,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Grünerløkka,1.000000,59.92433,10.76119,1,Coffee Shop,Bar,Park,Italian Restaurant,Burger Joint,Café,Cocktail Bar,Brewery,Pub,Asian Restaurant
1,Frogner,0.909149,59.91886,10.70239,1,Gourmet Shop,Italian Restaurant,Scandinavian Restaurant,Furniture / Home Store,Wine Shop,French Restaurant,Indian Restaurant,Movie Theater,Park,Cosmetics Shop
2,GamleOslo,0.891923,59.90624,10.77147,1,Coffee Shop,Record Shop,Park,Café,Plaza,Cocktail Bar,Italian Restaurant,Pizza Place,Restaurant,Moving Target
3,Nordstrand,0.712985,59.86075,10.80260,2,Grocery Store,Fast Food Restaurant,Gym,Supermarket,Coffee Shop,Restaurant,Bus Station,Shopping Mall,Juice Bar,Bookstore
4,NordreAker,0.709183,59.94933,10.73140,2,Grocery Store,Bus Station,Supermarket,Rental Car Location,Metro Station,Shopping Mall,Electronics Store,Kids Store,Fast Food Restaurant,Bike Shop
5,Østensjø,0.665370,59.87992,10.82584,4,Fast Food Restaurant,Grocery Store,Lake,Soccer Field,Yoga Studio,Cosmetics Shop,Creperie,Deli / Bodega,Department Store,Dessert Shop
6,VestreAker,0.646676,59.94805,10.64203,2,Metro Station,Grocery Store,Sushi Restaurant,Gym / Fitness Center,Café,Shopping Mall,Sporting Goods Shop,Gym,Coffee Shop,Bakery
7,Alna,0.636421,59.92680,10.84570,2,Electronics Store,Furniture / Home Store,Grocery Store,Supermarket,Department Store,Bus Station,Bookstore,Kids Store,Flower Shop,Sporting Goods Shop
8,Sagene,0.500691,59.93868,10.75763,1,Café,Sushi Restaurant,Park,Bakery,Deli / Bodega,Coffee Shop,Bus Station,Dessert Shop,Skate Park,Fish Market
9,SøndreNordstrand,0.327198,59.83342,10.80778,0,Grocery Store,Shopping Mall,Stadium,Fast Food Restaurant,Yoga Studio,Electronics Store,Convenience Store,Cosmetics Shop,Creperie,Deli / Bodega


It seems that Cluster 1 is the most important one for selecting the avenue

### 12. Visualize Clusters

In [43]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_oslo_merged['Latitude'], df_oslo_merged['Longitude'], df_oslo_merged['Borough'],df_oslo_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters.save('index-clusters.html')     
map_clusters

The epicentres are based on the number of Residents apparently, except for Sagene and St.H

Grunerlokka seems to be the place to expand the brand, but Frogner and Gamle Oslo are also good choices